In [1]:
# On Google colab:

# Data
!mkdir data
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
!mv input.txt data/shakespeare.txt

# Module
!pip install git+https://github.com/shuiruge/nanogpt.git

--2024-03-18 15:44:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-03-18 15:44:44 (19.9 MB/s) - ‘input.txt’ saved [1115394/1115394]

  Cloning https://github.com/shuiruge/nanogpt.git to /tmp/pip-req-build-5o3q876g
  Running command git clone --filter=blob:none --quiet https://github.com/shuiruge/nanogpt.git /tmp/pip-req-build-5o3q876g
  Resolved https://github.com/shuiruge/nanogpt.git to commit 23a263958d209b0ec4055aa76cad02c0f20e6539
  Preparing metadata (setup.py) ... done
  Created wheel for nanogpt: filename=nanogpt-0.0.1-py

In [2]:
# # Locally:
# import sys
# sys.path.append('../nanogpt')

In [3]:
import numpy as np
import tensorflow as tf
from keras.layers import Layer, Dense, LayerNormalization, Dropout
from keras.models import Model
from keras.optimizers import AdamW
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping
from dataclasses import dataclass
from typing import List

from nanogpt.utils import (
# from utils import (
    CharacterTokenizer, LanguageModelDataGenerator, TokPosEmbedding, FeedForward,
    ResNetWrapper, MultiHeadSelfAttention,
)

tf.random.set_seed(42)

In [4]:
with open('data/shakespeare.txt', 'r') as f:
    corpus = ''.join(f.readlines())
print(corpus[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
tokenizer = CharacterTokenizer(corpus)
tokenizer.vocab_size

65

In [6]:
seq_len = 64
data = LanguageModelDataGenerator(tokenizer.encode(corpus))

contexts = []
targets = []
context = None
while True:
    try:
        next_context, _ = data(seq_len, False)
        if context is None:
            context = next_context
            continue
        contexts.append(context)
        targets.append(next_context)
        context = next_context
    except StopIteration:
        break
contexts = np.stack(contexts).astype('int64')
targets = np.stack(targets).astype('int64')

In [7]:
contexts[:3], targets[:3]

(array([[18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14,
         43, 44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,
          1, 39, 52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43,
         39, 56,  1, 51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50],
        [47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
         44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1,
         39, 52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39,
         56,  1, 51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50],
        [56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
         53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
         52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,
          1, 51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10]]),
 array([[47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
         44, 53, 56, 43,  1, 61, 43,  1, 54, 5

In [8]:
# class CausalSelfAttention(Layer):
#     """A standard version of self-attention for GPT.

#     Args:
#         num_heads: int
#     """

#     def __init__(self, model_dim, num_heads):
#         super().__init__()
#         self.model_dim = model_dim
#         self.num_heads = num_heads

#         self.multihead = MultiHeadWrapper(self.num_heads)
#         self.get_query = Dense(self.model_dim)
#         self.get_key = Dense(self.model_dim)
#         self.get_value = Dense(self.model_dim)

#     def call(self, x, return_weights=False):
#         """
#         Args:
#             x: tf.Tensor
#                 Shape [..., seq_len, dim]
#             return_weights: bool
#                 If return the attention weights. Defaults to False.

#         Returns: tf.Tensor or (tf.Tensor, tf.Tensor)
#             If return_weights is false, then return the output only, which has
#             the same shape and dtype as the x. Otherwise, return the output as
#             well as the attention weights, which has shape
#             [..., num_heads, seq_len, seq_len].
#         """
#         query = self.get_query(x)
#         key = self.get_key(x)
#         value = self.get_value(x)

#         # Split the last dimension into multiple heads.
#         query = self.multihead.split_heads(query)
#         key = self.multihead.split_heads(key)
#         value = self.multihead.split_heads(value)

#         # Mask the self-communication.
#         seq_len = tf.shape(x)[-2]
#         # Like
#         # [[0., 1., 1., 1.],
#         #  [0., 0., 1., 1.],
#         #  [0., 0., 0., 1.],
#         #  [0., 0., 0., 0.]]
#         mask = 1 - tf.linalg.band_part(tf.ones([seq_len, seq_len]), -1, 0)

#         # The communication is implemented by a Luong-style attention.
#         output, attention_weights = luong_attention(query, key, value, mask)

#         # Concatenate the heads together.
#         output = self.multihead.concat_heads(output)

#         # The ResNet trick
#         return (output, attention_weights) if return_weights else output

In [9]:
class CausalSelfAttention(Layer):
    """A standard version of self-attention for GPT.

    Args:
        model_dim: int
        num_heads: int
    """

    def __init__(self, model_dim, num_heads):
        super().__init__()
        self.model_dim = model_dim
        self.num_heads = num_heads

        self.self_attention = MultiHeadSelfAttention(model_dim, num_heads)

    def call(self, x, return_weights=False):
        """
        Args:
            x: tf.Tensor
                Shape [..., seq_len, dim]
            return_weights: bool
                If return the attention weights. Defaults to False.

        Returns: tf.Tensor or (tf.Tensor, tf.Tensor)
            If return_weights is false, then return the output only, which has
            the same shape and dtype as the x. Otherwise, return the output as
            well as the attention weights, which has shape
            [..., num_heads, seq_len, seq_len].
        """
        seq_len = tf.shape(x)[-2]
        # Like
        # [[0., 1., 1., 1.],
        #  [0., 0., 1., 1.],
        #  [0., 0., 0., 1.],
        #  [0., 0., 0., 0.]]
        mask = 1 - tf.linalg.band_part(tf.ones([seq_len, seq_len]), -1, 0)
        return self.self_attention(x, mask, return_weights)

In [10]:
@dataclass
class GPTConfig:
    vocab_size: int
    seq_len: int
    embed_dim: int
    model_dim: int
    num_heads: int
    ffd_hidden_units: List[int]
    num_trans_blocks: int


class NanoGPT(Model):

    def __init__(self, cfg: GPTConfig):
        super().__init__()
        self.cfg = cfg

        self.embedding_layer = TokPosEmbedding(
            cfg.vocab_size, cfg.seq_len, cfg.embed_dim)

        # The so-called transformer-blocks.
        self.trans_blocks = []
        for _ in range(cfg.num_trans_blocks):
            self.trans_blocks.append(
                ResNetWrapper(CausalSelfAttention(cfg.model_dim, cfg.num_heads))
            )
            self.trans_blocks.append(
                ResNetWrapper(FeedForward(cfg.ffd_hidden_units, cfg.model_dim))
            )

        self.output_layer = Dense(cfg.vocab_size)

    def call(self, x):
        x = self.embedding_layer(x)
        for layer in self.trans_blocks:
            x = layer(x)
        x = self.output_layer(x)
        return x

    def generate(self, init_token_ids, num_new_tokens, T):
        """Generates new tokens from the initial.

        The "temperature" T controls the randomness, as in the Boltzmann
        distributions.

        Args:
            init_token_ids: List[int]
            num_new_tokens: int
            T: float

        Returns: List[int]
            It also includes the initial token-IDs. So, the length is the
            `len(initial_token_ids) +
        """
        init_token_ids = tf.convert_to_tensor(init_token_ids)

        # Add batch_size for matching the input shape of `self.call`.
        # [1, len(init_token_ids)]
        token_ids = tf.expand_dims(init_token_ids, axis=0)

        for _ in range(num_new_tokens):
            # [1, seq_len, vocab_size]
            logits = self(token_ids[:, -self.cfg.seq_len:])
            # We only use the last sequence element for output.
            # [1, vocab_size]
            logits = logits[:, -1, :]
            # [1, 1]
            next_token_id = tf.random.categorical(logits/T, 1)
            token_ids = tf.concat([token_ids, next_token_id], axis=1)

        # Drop the batch_size
        token_ids = tf.squeeze(token_ids, axis=0)
        return token_ids

In [11]:
# Andrej Karpathy's configuration.
# See: https://www.youtube.com/watch?v=kCc8FmEb1nY&t=6068s, at 1:40:30.
# This configuration will get a minimal validation loss about 1.48.
# And this configuration is not for our vanilla GPT, but Andrej's nano GPT,
# which is much more complicated than ours.
# cfg = GPTConfig(tokenizer.vocab_size, seq_len,
#                 embed_dim=384,
#                 model_dim=384,
#                 num_heads=6,
#                 ffd_hidden_units=[4*384],
#                 num_trans_blocks=6)

# We try a much much smaller one.
cfg = GPTConfig(tokenizer.vocab_size, seq_len,
                embed_dim=64,
                model_dim=64,
                num_heads=4,
                ffd_hidden_units=[4*64],
                num_trans_blocks=2)

model = NanoGPT(cfg)
model.build([None, seq_len])
model.summary()

Model: "nano_gpt"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tok_pos_embedding (TokPosE  multiple                  8256      
 mbedding)                                                       
                                                                 
 resnet_wrapper_of_causal_s  multiple                  8320      
 elf_attention (ResNetWrapp                                      
 er)                                                             
                                                                 
 resnet_wrapper_of_feed_for  multiple                  33216     
 ward (ResNetWrapper)                                            
                                                                 
 resnet_wrapper_of_causal_s  multiple                  8320      
 elf_attention_1 (ResNetWra                                      
 pper)                                                    

In [12]:
# model(contexts[:10])

In [13]:
model.compile(
    optimizer=AdamW(),
    loss=SparseCategoricalCrossentropy(from_logits=True),
)
model.fit(
    x=contexts,
    y=targets,
    batch_size=64,
    validation_split=0.1,
    # The epochs argument shall be as large as possible. And we control the
    # true epochs by early-stopping.
    epochs=100,
    callbacks=[EarlyStopping()]
)
# For our smaller configuration, the training will overfit after epoch 9.

Epoch 1/100
15685/15685 [==============================] - 228s 14ms/step - loss: 1.5771 - val_loss: 1.7034
Epoch 2/100
15685/15685 [==============================] - 198s 13ms/step - loss: 1.4334 - val_loss: 1.6972
Epoch 3/100
15685/15685 [==============================] - 201s 13ms/step - loss: 1.4036 - val_loss: 1.6803
Epoch 4/100
15685/15685 [==============================] - 197s 13ms/step - loss: 1.3874 - val_loss: 1.6926


In [18]:
generated = model.generate(contexts[0, :], 1000, 0.25)
print(tokenizer.decode(generated.numpy()))

First Citizen:
Before we proceed any further, hear me speak.

All:
What thou art thou hast and my father should be repose and the prince,
That thou hast have the people of the sent and with the heads
That the fire some of the king to the charged of the
To fair the did of the that should be so much a traitor
And the duke of the son of the best of the send of the people of the cold
That the langer of the sun, and the rest,
And that with the fire the commanded of the world and with
That the commanded of the respect and the world
The traitor house and the courteous and to the soul,
The people of the people of the rest of the hand
That the was the world to the prince,
And the seat the heads of his son and the king to the made of his death
The people of the soul, and the way the best
The state of the king of the wind of the world
That the manner of the world bed, and the seater
And by the people the soul, and will stay there the present of the world.

KING RICHARD III:
What says you the dear

In [19]:
generated = model.generate(contexts[0, :], 1000, 0.5)
print(tokenizer.decode(generated.numpy()))

First Citizen:
Before we proceed any further, hear me speak.

All:
The poster's if that less, and I will be parted against the queen of the
the courteous the senated stays to his so fair
Your purpose and look of the world false and blood;
So many crown to her by the place and courage thee,
Though the senate and of these my tongue and their to heart.

KING RICHARD III:
What is the pardon'd and love the seater that the sense that respectses
The secure in the cannot in the was hath the venge of the fire
And with his a commanded and and dead,
I am the rest the law of the people the the life,
But I am I make a part to such all the father been thoughts,
Which is the fairest say is in hearing son.
There should with his friends to thee with the sent to heaven,
Because and more stood of the head,
And the interce of the loved against the rest,
The all be stood reached in my counted
In promised the the face of the that shame time
To kings to light the great sings wretch
That he did bear the best 

In [20]:
generated = model.generate(contexts[0, :], 1000, 1)
print(tokenizer.decode(generated.numpy()))

First Citizen:
Before we proceed any further, hear me speak.

All my father speaknd execution, what I
Grlook to thee.

ELBOW:
Here comes heads,
Will bed theset my soul; and your blood punish'd,
When he nourage hide slired underfumerlesse dent and land.

PAPULET:
I shall see as though oes in the sacle:
Anon, my master, which if then parks myself,
That throw state fly piece we, I
say; not I beseech in to approach, sir:
But Gemio, my lord.

MORD Richard,
Whereight it for his life.

LADY ANNE:
O, not get, and one new shame; and
behold her gage.

CLARENCE:
Though that down my lord?

WARWICK:
And was it servant, this mother
Were to-morrow, singling held Bolingbroke
Lenduct quick the lothrew of his child
By you rainted now and Born? his by seen;
The bury, we another of themies, swelt his ground,
And proffer. I wister'd, have did
certain'd the look may hungrying; I have none committed;
They down either proud of reful crowning manulto'd death.

ANGELO:
To faith, thou, and there the outward poss